In [92]:

from openai import OpenAI
import json
from pykrx import stock
from pykrx import bond

client = OpenAI()

print("주식 정보를 분석해드려요. 날짜와 회사명을 말씀해주세요.")

주식 정보를 분석해드려요. 날짜와 회사명을 말씀해주세요.


In [93]:
#종목별 DIV/BPS/PER/EPS 조회
def get_market_fundamental(start_dt, end_dt, ticket):
    df = stock.get_market_fundamental(start_dt, end_dt, ticket)
    return json.dumps(df.head(10000).to_dict(orient="records"), ensure_ascii=False)

#일자별 OHLCV 조회
def get_market_ohlcv(start_dt, end_dt, ticket):
    df = stock.get_market_ohlcv(start_dt, end_dt, ticket)
    return json.dumps(df.head(10000).to_dict(orient="records"), ensure_ascii=False)

In [94]:
tickers = stock.get_market_ticker_list()
종목 = {}

for ticker in tickers:
    종목[stock.get_market_ticker_name(ticker)] = ticker

def get_stock_info(start_dt, end_dt, ticker):
    data = {}
    data["fundamental"] = get_market_fundamental(start_dt, end_dt, ticker)
    data["ohlcv"] = get_market_ohlcv(start_dt, end_dt, ticker)

    return str(data)

messages = [{"role": "user", "content": "2024년 1월 1일 ~ 2024년 1월 21일의 LG전자 주식 정보를 요약해줘"}]
functions = [
    {
        "name": "get_stock_info",
        "description": "주식 정보를 제공받습니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "start_dt": {
                    "type": "string",
                    "description": "시작일자, ex) 2024년이라고 말한 경우 2024년 1월 1일, 2024년 5월이라고 말한 경우 2024년 5월 1일",
                },
                "end_dt": {
                    "type": "string",
                    "description": "종료일자, ex) 2024년이라고 말한 경우 2024년 1월 1일, 2024년 5월이라고 말한 경우 2024년 5월 1일",
                },
                "ticker": {
                    "type": "string", 
                    "description": str(종목)
                },
            },
            "required": ["start_dt", "end_dt", "ticker"],
        },
    }
]

In [95]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"start_dt":"2024-01-01","end_dt":"2024-01-21","ticker":"066570"}', name='get_stock_info'), tool_calls=None)


In [96]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_stock_info": get_stock_info,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        start_dt=function_args['start_dt'],
        end_dt=function_args['end_dt'],
        ticker=function_args['ticker'],
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    print(second_response.choices[0].message.content)

2024년 1월 1일부터 1월 21일까지의 LG전자(주식 코드: 066570) 주식 정보 요약은 다음과 같습니다:

### 가격 변동
- **1월 1일**: 시가 101,500원, 종가 101,400원 (등락률: -0.39%)
- **1월 2일**: 시가 100,400원, 종가 99,500원 (등락률: -1.87%)
- **1월 3일**: 시가 98,500원, 종가 97,800원 (등락률: -1.71%)
- **1월 4일**: 시가 97,600원, 종가 97,100원 (등락률: -0.72%)
- **1월 5일**: 시가 97,500원, 종가 99,100원 (등락률: +2.06%)
- **1월 8일**: 시가 100,100원, 종가 98,000원 (등락률: -1.11%)
- **1월 9일**: 시가 98,000원, 종가 97,200원 (등락률: -0.82%)
- **1월 10일**: 시가 97,500원, 종가 95,000원 (등락률: -2.26%)
- **1월 11일**: 시가 95,000원, 종가 94,700원 (등락률: -0.32%)
- **1월 12일**: 시가 95,300원, 종가 95,100원 (등락률: +0.42%)
- **1월 15일**: 시가 94,700원, 종가 94,600원 (등락률: -0.53%)
- **1월 16일**: 시가 94,000원, 종가 91,300원 (등락률: -3.49%)
- **1월 17일**: 시가 91,100원, 종가 91,200원 (등락률: -0.11%)
- **1월 18일**: 시가 92,200원, 종가 91,800원 (등락률: +0.66%)

### 주가 통계
- **고가**: 101,800원 (1월 1일)
- **저가**: 90,700원 (1월 16일)
- **거래량**: 각 일별 평균적으로 400,000 ~ 800,000주 거래됨

### 주요 재무 지표 (2024년 기준)
- **BPS (주당자산가치)**: 105,473원
- **PER (주가수익비율)**: 13.75 ~ 15.27
- **PBR (주가순자산비율